# **Data Augmentation with (Conditional) Gan** 

**The objective is to verify if a dataset of synthetic images generated from a GAN can be used to effectively train a classifier, and how it compares to training the classifier with the original training set.**
**Verify if synthetic images can serve as additional data in addition to the original training set.**

Dataset - **CIFAR10**


In [5]:
import tensorflow as tf
import numpy as np

# Loading the CIFAR-10 dataset 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# --- Preprocessing ---
# Normalize pixel values from the [0, 255] range to the [-1, 1] range.
# This is a common practice for GANs as it helps the generator's
# output (using a tanh activation) match the real image distribution.
x_train = (x_train.astype('float32') - 127.5) / 127.5

# Print the shape of the training data to confirm
print("Shape of training images:", x_train.shape)
print("Shape of training labels:", y_train.shape)

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # If GPUs are found, TensorFlow will automatically use them
  print(f"✅ GPU(s) found: {len(gpus)}")
  for gpu in gpus:
    print(f"  - {gpu}")
else:
  # If no GPUs are found, TensorFlow will use the CPU
  print("❌ No GPU found. TensorFlow will use the CPU.")

Shape of training images: (50000, 32, 32, 3)
Shape of training labels: (50000, 1)
❌ No GPU found. TensorFlow will use the CPU.


#### Building the Discriminator 

In [6]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, Embedding
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define input shapes
IMG_SHAPE = (32, 32, 3)
NUM_CLASSES = 10

def build_discriminator():
    # Input for the image
    img_input = Input(shape=IMG_SHAPE)

    # Input for the class label
    label_input = Input(shape=(1,))
    
    # Convert label into a dense vector and reshape to match the image dimensions
    label_embedding = Embedding(NUM_CLASSES, 50)(label_input)
    label_embedding = Dense(IMG_SHAPE[0] * IMG_SHAPE[1])(label_embedding)
    label_embedding = Reshape((IMG_SHAPE[0], IMG_SHAPE[1], 1))(label_embedding)

    # Combine the label embedding and the image
    concatenated_input = Concatenate()([img_input, label_embedding])

    # CNN layers to classify the input
    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(concatenated_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Flatten()(x)
    
    # Output layer: a single value indicating real (1) or fake (0)
    x = Dense(1, activation='sigmoid')(x)

    # Create and compile the discriminator model
    discriminator = Model([img_input, label_input], x, name="discriminator")
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    
    return discriminator

discriminator = build_discriminator()
discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 50)     │        500 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1, 1024)   │     52,224 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 32, 32, 1) │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32, 32, 4) │          0 │ input_layer_2[0]… │
│ (Concatenate)       │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 16,    │      2,368 │ concatenate_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 16, 16,    │          0 │ conv2d_2[0][0]    │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 8, 8, 128) │     73,856 │ leaky_re_lu_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_3       │ (None, 8, 8, 128) │          0 │ conv2d_3[0][0]    │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8192)      │          0 │ leaky_re_lu_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │      8,193 │ flatten_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 137,141 (535.71 KB)

 Trainable params: 137,141 (535.71 KB)

 Non-trainable params: 0 (0.00 B)

#### Building the Generator

In [7]:
# Define the size of the random noise vector
LATENT_DIM = 100

def build_generator():
    # Input for the random noise
    noise_input = Input(shape=(LATENT_DIM,))

    # Input for the class label
    label_input = Input(shape=(1,))
    
    # Process the label, reshaping it for combination with the noise
    label_embedding = Embedding(NUM_CLASSES, 50)(label_input)
    label_embedding = Dense(8 * 8)(label_embedding)
    label_embedding = Reshape((8, 8, 1))(label_embedding)

    # Process the noise into a small feature map
    noise = Dense(128 * 8 * 8, activation='relu')(noise_input)
    noise = Reshape((8, 8, 128))(noise)

    # Combine the processed noise and label
    concatenated_input = Concatenate()([noise, label_embedding])

    # Upsample the combined input to a full-sized image using transposed convolutions
    x = Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu')(concatenated_input)
    x = Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', activation='relu')(x)
    
    # Output layer: creates a 32x32 image with 3 color channels (RGB)
    # The 'tanh' activation scales the output to [-1, 1], matching our preprocessed real images
    x = Conv2D(3, kernel_size=5, padding='same', activation='tanh')(x)

    # Create the generator model
    generator = Model([noise_input, label_input], x, name="generator")
    return generator

generator = build_generator()
generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 50)     │        500 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 8192)      │    827,392 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1, 64)     │      3,264 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 8, 8, 128) │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 8, 8, 1)   │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 8, 8, 129) │          0 │ reshape_3[0][0],  │
│ (Concatenate)       │                   │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 16, 16,    │    264,320 │ concatenate_2[0]… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 32, 32,    │    262,272 │ conv2d_transpose… │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32, 3) │      9,603 │ conv2d_transpose… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,367,351 (5.22 MB)

 Trainable params: 1,367,351 (5.22 MB)

 Non-trainable params: 0 (0.00 B)

#### Building and Compiling the cGAN

In [8]:
# When training the combined model, we only want to update the generator's weights.
# So, we set the discriminator to be non-trainable.
discriminator.trainable = False

# Model inputs
noise_input = Input(shape=(LATENT_DIM,))
label_input = Input(shape=(1,))

# The generator produces an image from the inputs
generated_img = generator([noise_input, label_input])

# The discriminator evaluates the generated image
validity = discriminator([generated_img, label_input])

# Create the combined model that links the generator to the discriminator
cgan = Model([noise_input, label_input], validity, name="cgan")
cgan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
cgan.summary()

Model: "cgan"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_7       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ generator           │ (None, 32, 32, 3) │  1,367,351 │ input_layer_6[0]… │
│ (Functional)        │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ discriminator       │ (None, 1)         │    137,141 │ generator[0][0],  │
│ (Functional)        │                   │            │ input_layer_7[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,504,492 (5.74 MB)

 Trainable params: 1,367,351 (5.22 MB)

 Non-trainable params: 137,141 (535.71 KB)

#### Training Loop

In [9]:
import matplotlib.pyplot as plt

def sample_images(epoch):
    """A helper function to generate and display a grid of images for visual inspection."""
    r, c = 2, 5
    noise = np.random.normal(0, 1, (r * c, LATENT_DIM))
    # Generate one image for each class
    sampled_labels = np.arange(0, 10).reshape(-1, 1)
    
    gen_imgs = generator.predict([noise, sampled_labels])
    
    # Rescale images from [-1, 1] to [0, 1] for plotting
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c, figsize=(10,4))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].set_title(f"Class: {sampled_labels[cnt][0]}")
            axs[i,j].axis('off')
            cnt += 1
    plt.show()
    plt.close()

def train_cgan(epochs, batch_size=128, sample_interval=1000):
    # Ground truth labels for real (1) and fake (0) images
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        # Select a random batch of real images and their labels
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_imgs, labels = x_train[idx], y_train[idx]

        # Generate a batch of fake images with the same labels
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        gen_imgs = generator.predict([noise, labels])

        # Train the discriminator on separate batches of real and fake images
        d_loss_real = discriminator.train_on_batch([real_imgs, labels], valid)
        d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # -----------------
        #  Train Generator
        # -----------------
        # Generate new noise and a batch of RANDOM labels to train the generator
        noise = np.random.normal(0, 1, (batch_size, LATENT_DIM))
        sampled_labels = np.random.randint(0, NUM_CLASSES, batch_size).reshape(-1, 1)

        # Train the generator (via the combined cgan model) to make the discriminator think the fake images are real
        g_loss = cgan.train_on_batch([noise, sampled_labels], valid)

        # Print progress and show sample images at intervals
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")
        
        if epoch % sample_interval == 0:
            sample_images(epoch)

# To start training (this will take a long time):
# train_cgan(epochs=20000, batch_size=64, sample_interval=1000)